# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about vision APIs for vision-language models and embedding APIs for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30020 --host 0.0.0.0"
)

wait_for_server("http://localhost:30020")

[2025-02-09 18:50:22] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=30020, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=85521530, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_

[2025-02-09 18:50:41 TP0] Init torch distributed begin.
[2025-02-09 18:50:41 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-09 18:50:42 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.27it/s]


...

 96%|█████████▌| 22/23 [00:04<00:00,  5.18it/s]

100%|██████████| 23/23 [00:05<00:00,  4.46it/s]
[2025-02-09 18:50:50 TP0] Capture cuda graph end. Time elapsed: 5.17 s


[2025-02-09 18:50:51 TP0] max_total_num_tokens=444372, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2025-02-09 18:50:51] INFO:     Started server process [3583923]
[2025-02-09 18:50:51] INFO:     Waiting for application startup.
[2025-02-09 18:50:51] INFO:     Application startup complete.
[2025-02-09 18:50:51] INFO:     Uvicorn running on http://0.0.0.0:30020 (Press CTRL+C to quit)


[2025-02-09 18:50:52] INFO:     127.0.0.1:56280 - "GET /v1/models HTTP/1.1" 200 OK
[2025-02-09 18:50:52] INFO:     127.0.0.1:56292 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-09 18:50:52 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-09 18:50:54] INFO:     127.0.0.1:56304 - "POST /generate HTTP/1.1" 200 OK
[2025-02-09 18:50:54] The server is fired up and ready to roll!


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30020/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-02-09 18:50:58 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, cache hit rate: 2.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-09 18:50:58 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 5.57, #queue-req: 0
[2025-02-09 18:50:58] INFO:     127.0.0.1:56314 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-02-09 18:50:58 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, cache hit rate: 20.63%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-09 18:50:58 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.


[2025-02-09 18:50:59 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.00, gen throughput (token/s): 126.64, #queue-req: 0


[2025-02-09 18:50:59 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.00, gen throughput (token/s): 142.64, #queue-req: 0


[2025-02-09 18:50:59 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.00, gen throughput (token/s): 142.57, #queue-req: 0
[2025-02-09 18:50:59] INFO:     127.0.0.1:56314 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-02-09 18:50:59] INFO:     127.0.0.1:56314 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-02-09 18:50:59 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, cache hit rate: 33.73%, token usage: 0.00, #running-req: 0, #queue-req: 0
This is only a test

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-02-09 18:50:59 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 32.57%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-09 18:50:59 TP0] Decode batch. #running-req: 1, #token: 25, token usage: 0.00, gen throughput (token/s): 129.78, #queue-req: 0


[2025-02-09 18:51:00 TP0] Decode batch. #running-req: 1, #token: 65, token usage: 0.00, gen throughput (token/s): 148.03, #queue-req: 0
[2025-02-09 18:51:00] INFO:     127.0.0.1:56314 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-02-09 18:51:00 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 31.35%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-09 18:51:00 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.
[2025-02-09 18:51:00] INFO:     127.0.0.1:56314 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30020/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-02-09 18:51:00] INFO:     127.0.0.1:56324 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-09 18:51:00] INFO:     127.0.0.1:56324 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-09 18:51:00 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, cache hit rate: 44.53%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-02-09 18:51:00 TP0] Decode batch. #running-req: 1, #token: 54, token usage: 0.00, gen throughput (token/s): 104.54, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-02-09 18:51:03] INFO:     127.0.0.1:56324 - "GET /v1/batches/batch_617a0455-7a5e-40f2-a743-d01be02fb26b HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-02-09 18:51:03] INFO:     127.0.0.1:56324 - "GET /v1/files/backend_result_file-f2fcdb95-b7a4-4c19-9b13-0a7c730ade2b/content HTTP/1.1" 200 OK


[2025-02-09 18:51:03] INFO:     127.0.0.1:56324 - "DELETE /v1/files/backend_result_file-f2fcdb95-b7a4-4c19-9b13-0a7c730ade2b HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30020/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-02-09 18:51:03] INFO:     127.0.0.1:33448 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-09 18:51:03] INFO:     127.0.0.1:33448 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-09 18:51:03 TP0] Prefill batch. #new-seq: 5, #new-token: 150, #cached-token: 125, cache hit rate: 45.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-09 18:51:03 TP0] Prefill batch. #new-seq: 95, #new-token: 2850, #cached-token: 2375, cache hit rate: 45.41%, token usage: 0.00, #running-req: 5, #queue-req: 0


[2025-02-09 18:51:03 TP0] Decode batch. #running-req: 100, #token: 3525, token usage: 0.01, gen throughput (token/s): 144.50, #queue-req: 0


[2025-02-09 18:51:04 TP0] Decode batch. #running-req: 100, #token: 7525, token usage: 0.02, gen throughput (token/s): 12087.51, #queue-req: 0


[2025-02-09 18:51:04 TP0] Decode batch. #running-req: 100, #token: 11525, token usage: 0.03, gen throughput (token/s): 11815.59, #queue-req: 0


...

[2025-02-09 18:51:13] INFO:     127.0.0.1:59694 - "GET /v1/batches/batch_56fd9554-d96e-4dfc-af18-e9657054fce0 HTTP/1.1" 200 OK


[2025-02-09 18:51:16] INFO:     127.0.0.1:59694 - "GET /v1/batches/batch_56fd9554-d96e-4dfc-af18-e9657054fce0 HTTP/1.1" 200 OK


[2025-02-09 18:51:19] INFO:     127.0.0.1:59694 - "GET /v1/batches/batch_56fd9554-d96e-4dfc-af18-e9657054fce0 HTTP/1.1" 200 OK


[2025-02-09 18:51:22] INFO:     127.0.0.1:59694 - "GET /v1/batches/batch_56fd9554-d96e-4dfc-af18-e9657054fce0 HTTP/1.1" 200 OK


[2025-02-09 18:51:25] INFO:     127.0.0.1:59694 - "GET /v1/batches/batch_56fd9554-d96e-4dfc-af18-e9657054fce0 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url="http://127.0.0.1:30020/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-02-09 18:51:28] INFO:     127.0.0.1:43860 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-09 18:51:28] INFO:     127.0.0.1:43860 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-09 18:51:28 TP0] Prefill batch. #new-seq: 6, #new-token: 6, #cached-token: 324, cache hit rate: 48.27%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-09 18:51:28 TP0] Prefill batch. #new-seq: 85, #new-token: 85, #cached-token: 4590, cache hit rate: 69.94%, token usage: 0.01, #running-req: 6, #queue-req: 0
[2025-02-09 18:51:28 TP0] Prefill batch. #new-seq: 282, #new-token: 8192, #cached-token: 7311, cache hit rate: 56.50%, token usage: 0.01, #running-req: 91, #queue-req: 127


[2025-02-09 18:51:28 TP0] Prefill batch. #new-seq: 128, #new-token: 3817, #cached-token: 3175, cache hit rate: 54.17%, token usage: 0.03, #running-req: 372, #queue-req: 1


[2025-02-09 18:51:29 TP0] Decode batch. #running-req: 500, #token: 27525, token usage: 0.06, gen throughput (token/s): 636.35, #queue-req: 0


[2025-02-09 18:51:30 TP0] Decode batch. #running-req: 500, #token: 47525, token usage: 0.11, gen throughput (token/s): 26538.74, #queue-req: 0


...

[2025-02-09 18:51:36 TP0] Decode batch. #running-req: 500, #token: 187525, token usage: 0.42, gen throughput (token/s): 19488.38, #queue-req: 0


[2025-02-09 18:51:37 TP0] Decode batch. #running-req: 500, #token: 207525, token usage: 0.47, gen throughput (token/s): 18893.96, #queue-req: 0


[2025-02-09 18:51:38] INFO:     127.0.0.1:34802 - "POST /v1/batches/batch_42327c4d-1496-4ea4-bcc3-038b0972aa27/cancel HTTP/1.1" 200 OK


[2025-02-09 18:51:41] INFO:     127.0.0.1:34802 - "GET /v1/batches/batch_42327c4d-1496-4ea4-bcc3-038b0972aa27 HTTP/1.1" 200 OK


[2025-02-09 18:51:41] INFO:     127.0.0.1:34802 - "DELETE /v1/files/backend_input_file-7fa3139b-27f3-4ff3-a6a6-17ff16c476e8 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)